In [ ]:
import sys
sys.path.insert(0, '../')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
actas_con_personas = pd.read_csv('../../../assets/silver/data_utilizada/personas_un_acta_por_fila.csv')
actas_con_personas['fecha'] = pd.to_datetime(actas_con_personas['fecha'], format='%Y-%m-%d')
actas_con_personas['dni'] = actas_con_personas['dni'].astype(str)
actas_con_personas.drop(columns = ['Unnamed: 0'], inplace = True)

In [ ]:
actas_con_personas['dni'].nunique()

Siguiendo la misma logica que con el CBC, debemos tener las siguientes columnas:

Tenemos una fila por DNI y, como ahora definimos que queremos predecir qué pasa en el semestre relativo 4, tendríamos por cada semestre relativo j (j tomando valores 0 a 3 inclusive) las columnas:

* #inscripciones_j: como la que usamos para la target
* #TPs_aprobados_j: Para los TPS (excepto casos específicos) no tenemos nota numérica, solamente si aprobó, reprobó o dejó la materia, por eso tomamos la cantidad. En cambio para finales si tenemos el dato de la nota.
* #finales_inscriptos_j: como la que usamos para calcular la target
* #Promedio_de_finales_j: Si no rindió ninguno es 0. Si se anotó y no se presentó suma 0 en el cálculo del promedio.

Hay que agregar las materias como columnas. En principio son 40 para la carrera.
La difrencia es que en el cbc es un cluster y en la carrera se ponen en el orden que haya rendido. 
Para cada materia:
hay que tomar fecha 0 como 1/1/2020 y a partir de ahi un numero natural que sean la cantidad de dias desde 1/1/2020. Luego, agregar la fecha de TP. agregar la nota tambien.  Esto es para materias aprobadas.

La idea es tener una distribucion empirica de que rinde, cada cuanto.
 *Si alguien rindio mas de 40 materias se toman las 1ras 40. Vemos si hay mucha gente en estas condiciones.

Entonces por cada una de las 40 materias en los semestres relativos 0 a 3 (me fijo si hay gente con más materias que eso y lo dejo en el docs) tenemos:
- 'fecha_TP_materia_x' donde es un número que cuenta los días desde 1/1/2020
- 'tp_aprobado': 1 si aprobó 0 si no?
- 'fecha_final_materia_x' donde es un número que cuenta los días desde 1/1/2020
- 'nota_final_materia_x'

Donde x va de 1 a 40 y si rindió la materia más de una vez, tenemos columnas distintas informando sobre las distintas veces que cursó.

Doy un ejemplo: Labo de datos TP aprobado el 1/2/2022 y final aprobado con 8 1/3/2022
- 'fecha_TP_materia_x' = 31
- 'tp_aprobado': 1
- 'fecha_final_materia_x' = 31+28 = 59
- 'nota_final_materia_x' = 8
Si no hubiera aprobado la materia, Labo de datos TP desaprobado/abandonado el 1/2/2022
- 'fecha_TP_materia_x' = 31
- 'tp_aprobado': 0
- 'fecha_final_materia_x' = dato faltante
- 'nota_final_materia_x' = 0

Vemos la cantidad de materias que hicieron las personas. Como agrupo cada materia con su final y, en caso de cursar la materia más de una vez tengo conjuntos de columnas distintos. Debo contar la cantidad de filas que cada DNI tiene de 'Acta de Regulares/Promociones'

In [ ]:
actas_con_personas.columns

In [ ]:
actas_con_personas[actas_con_personas['tipo_acta'] == 'Acta de Regulares/Promociones'].groupby('dni').size().reset_index(name='count').sort_values(by=['count'], ascending=False)

In [ ]:
# Crear el DataFrame filtrado y agrupado
df_counts = (
    actas_con_personas[actas_con_personas['tipo_acta'] == 'Acta de Regulares/Promociones']
    .groupby('dni')
    .size()
    .reset_index(name='count')
)

# Contar cuántas veces aparece cada valor de 'count'
count_distribution = df_counts['count'].value_counts().sort_index()

plt.figure(figsize=(8, 5))
sns.barplot(x=count_distribution.index, y=count_distribution.values, color='skyblue')
plt.xlabel('Cantidad de actas por DNI')
plt.ylabel('Cantidad de personas')
plt.title('Distribución de cantidad de actas de Regulares/Promociones por persona')
plt.show()

In [ ]:
# Contar cuántas veces aparece cada valor de 'count' y calcular proporción
count_distribution = df_counts['count'].value_counts(normalize=True).sort_index()

plt.figure(figsize=(8, 5))
sns.barplot(x=count_distribution.index, y=count_distribution.values, color='skyblue')
plt.xlabel('Cantidad de actas por DNI')
plt.ylabel('Proporción de personas')
plt.title('Distribución proporcional de actas de Regulares/Promociones por persona')

plt.show()

In [ ]:
# Contar cuántas personas tienen cada cantidad de actas
counts = df_counts['count'].value_counts().sort_index()

# Calcular el acumulado
cumulative_counts = counts.cumsum()

# Graficar
plt.figure(figsize=(8, 5))
plt.plot(cumulative_counts.index, cumulative_counts.values, marker='o', color='steelblue')
plt.xlabel('Cantidad de actas por persona')
plt.ylabel('Cantidad acumulada de personas')
plt.title('Distribución acumulada de actas de Regulares/Promociones por persona')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Calcular proporciones acumuladas
cumulative_proportions = (df_counts['count'].value_counts(normalize=True)
                          .sort_index()
                          .cumsum())

# Graficar proporciones acumuladas
plt.figure(figsize=(8, 5))
plt.plot(cumulative_proportions.index, cumulative_proportions.values, marker='o', color='darkorange')
plt.xlabel('Cantidad de actas por persona')
plt.ylabel('Proporción acumulada de personas')
plt.title('Distribución proporcional acumulada de actas de Regulares/Promociones por persona')
plt.grid(True)
plt.tight_layout()
# Agregar xticks para cada valor posible
plt.xticks(cumulative_proportions.index)
plt.show()


In [ ]:
cumulative_proportions

No es tan loco, pensemos que en la FCEN las carreras estan pensadas para que hagas 2 o 3 materias por cuatrimestre y nosotros estamos tomando 4 semestres relativos. Sin embargo, a su vez quiero ver qué pasa si saco las materias que sean del semestre relativo 4

In [ ]:
# Crear el DataFrame filtrado y agrupado
df_counts = (
    actas_con_personas[(actas_con_personas['tipo_acta'] == 'Acta de Regulares/Promociones') & (actas_con_personas['semestre_relativo'] != 4)]
    .groupby('dni')
    .size()
    .reset_index(name='count')
)

# Contar cuántas personas tienen cada cantidad de actas
counts = df_counts['count'].value_counts().sort_index()

# Calcular el acumulado
cumulative_counts = counts.cumsum()

# Graficar
plt.figure(figsize=(8, 5))
plt.plot(cumulative_counts.index, cumulative_counts.values, marker='o', color='steelblue')
plt.xlabel('Cantidad de actas por persona')
plt.ylabel('Cantidad acumulada de personas')
plt.title('Distribución acumulada de actas de Regulares/Promociones por persona')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Calcular proporciones acumuladas
cumulative_proportions = (df_counts['count'].value_counts(normalize=True)
                          .sort_index()
                          .cumsum())

# Graficar proporciones acumuladas
plt.figure(figsize=(8, 5))
plt.plot(cumulative_proportions.index, cumulative_proportions.values, marker='o', color='darkorange')
plt.xlabel('Cantidad de actas por persona')
plt.ylabel('Proporción acumulada de personas')
plt.title('Distribución proporcional acumulada de actas de Regulares/Promociones por persona')
plt.grid(True)
plt.tight_layout()
# Agregar xticks para cada valor posible
plt.xticks(cumulative_proportions.index)
plt.show()


In [ ]:
cumulative_proportions

In [ ]:
reverse_cumulative_counts

In [ ]:
# Calcular la cantidad de personas con cierta cantidad de actas o más (distribución acumulada inversa)
reverse_cumulative_counts = counts[::-1].cumsum()[::-1]

# También calculamos la proporción acumulada inversa
reverse_cumulative_proportions = (df_counts['count'].value_counts(normalize=True)
                                  .sort_index(ascending=False)
                                  .cumsum()
                                  .sort_index())

# Graficar proporción acumulada inversa
plt.figure(figsize=(8, 5))
plt.plot(reverse_cumulative_proportions.index, reverse_cumulative_proportions.values, marker='o', color='seagreen')
plt.xlabel('Cantidad de actas por persona')
plt.ylabel('Proporción de personas con esa cantidad o más')
plt.title('Proporción de personas con cierta cantidad de actas o más')
plt.grid(True)
plt.xticks(reverse_cumulative_proportions.index)
plt.tight_layout()
plt.show()

In [ ]:
reverse_cumulative_proportions

In [ ]:
# Crear el DataFrame filtrado y agrupado
df_counts = (
    actas_con_personas[(actas_con_personas['tipo_acta'] == 'Acta de Regulares/Promociones')]
    .groupby('dni')
    .size()
    .reset_index(name='count')
)

# Contar cuántas personas tienen cada cantidad de actas
counts = df_counts['count'].value_counts().sort_index()


# Calcular la cantidad de personas con cierta cantidad de actas o más (distribución acumulada inversa)
reverse_cumulative_counts = counts[::-1].cumsum()[::-1]

# También calculamos la proporción acumulada inversa
reverse_cumulative_proportions = (df_counts['count'].value_counts(normalize=True)
                                  .sort_index(ascending=False)
                                  .cumsum()
                                  .sort_index())

# Graficar proporción acumulada inversa
plt.figure(figsize=(8, 5))
plt.plot(reverse_cumulative_proportions.index, reverse_cumulative_proportions.values, marker='o', color='seagreen')
plt.xlabel('Cantidad de actas por persona')
plt.ylabel('Proporción de personas con esa cantidad o más')
plt.title('Proporción de personas con cierta cantidad de actas o más')
plt.grid(True)
plt.xticks(reverse_cumulative_proportions.index)
plt.tight_layout()
plt.show()

In [ ]:
reverse_cumulative_proportions

In [ ]:
df_counts[df_counts['count'] == 28]

In [ ]:
# Crear el DataFrame filtrado y agrupado
df_counts = (
    actas_con_personas
    .groupby('dni')
    .size()
    .reset_index(name='count')
)

# Contar cuántas personas tienen cada cantidad de actas
counts = df_counts['count'].value_counts().sort_index()


# Calcular la cantidad de personas con cierta cantidad de actas o más (distribución acumulada inversa)
reverse_cumulative_counts = counts[::-1].cumsum()[::-1]

# También calculamos la proporción acumulada inversa
reverse_cumulative_proportions = (df_counts['count'].value_counts(normalize=True)
                                  .sort_index(ascending=False)
                                  .cumsum()
                                  .sort_index())

# Graficar proporción acumulada inversa
plt.figure(figsize=(8, 5))
plt.plot(reverse_cumulative_proportions.index, reverse_cumulative_proportions.values, marker='o', color='seagreen')
plt.xlabel('Cantidad de actas por persona')
plt.ylabel('Proporción de personas con esa cantidad o más')
plt.title('Proporción de personas con cierta cantidad de actas o más')
plt.grid(True)
plt.xticks(reverse_cumulative_proportions.index)
plt.tight_layout()
plt.show()

In [ ]:
reverse_cumulative_proportions

In [ ]:
# Paso 1: Agrupar por dni y semestre_relativo
grupo = actas_con_personas.groupby(['dni', 'semestre_relativo'])['tipo_acta']

# Paso 2: Verificar si todos los tipo_acta dentro del grupo son 'Acta Examen'
solo_acta_examen = grupo.apply(lambda x: (x == 'Acta de Examen').all())

# Paso 3: Filtrar los grupos donde eso se cumple
resultado = solo_acta_examen[solo_acta_examen].reset_index()

# Ahora resultado contiene las combinaciones de dni y semestre_relativo donde solo hubo 'Acta Examen'


In [ ]:
resultado[resultado['semestre_relativo'] == 1]

In [ ]:
chusmeo = actas_con_personas[actas_con_personas['dni'].isin(resultado[resultado['semestre_relativo'] == 0]['dni'].unique())][['dni', 'materia', 'tipo_acta', 'resultado', 'nota', 'fecha', 'semestre_relativo']]

In [ ]:
chusmeo[chusmeo['semestre_relativo'] == 0].sort_values(by=['dni', 'fecha'], ascending=False)

In [ ]:
chusmeo = actas_con_personas[actas_con_personas['dni'].isin(resultado[resultado['semestre_relativo'] == 1]['dni'].unique())][['dni', 'materia', 'tipo_acta', 'resultado', 'nota', 'fecha', 'semestre_relativo']]
chusmeo[chusmeo['semestre_relativo'].isin([0,1])].sort_values(by=['dni', 'fecha'], ascending=False)

In [ ]:
# Paso 1: Filtramos columnas necesarias
df_filtrado = actas_con_personas[actas_con_personas['semestre_relativo'] != 4][['dni', 'materia', 'tipo_acta']]

# Paso 2: Agrupar por dni y materia
def tiene_examen_sin_regular(x):
    tipos = set(x)
    return 'Acta de Examen' in tipos and 'Acta de Regulares/Promociones' not in tipos

condicion = df_filtrado.groupby(['dni', 'materia'])['tipo_acta'].apply(tiene_examen_sin_regular)

# Paso 3: Filtrar los casos que cumplen
resultado = condicion[condicion].reset_index()

# resultado ahora tiene los dni y materias que cumplen la condición

In [ ]:
# Paso 1: Obtener la cantidad de veces que aparece cada DNI
frecuencias = resultado.groupby('dni').size()

# Paso 2: Agrupar por tamaño (por ejemplo, cuántos DNIs aparecen 1, 2, 3 veces...)
distribucion = frecuencias.value_counts().sort_index()

# Paso 3: Mostrar en tabla (opcional)
print(distribucion)

# Paso 4: Graficar
distribucion.plot(kind='bar')
plt.xlabel('Cantidad de veces que aparece un DNI')
plt.ylabel('Cantidad de DNIs')
plt.title('Distribución de tamaño de grupos por DNI')
plt.show()

In [ ]:
# Contar la cantidad de materias distintas por DNI
materias_por_dni = actas_con_personas.groupby('dni')['materia'].nunique().reset_index()

# Renombrar la columna para mayor claridad
materias_por_dni.columns = ['dni', 'materias_distintas']

print(materias_por_dni)

In [ ]:
# Definimos un rango máximo para las materias distintas que querés analizar
max_materias = materias_por_dni['materias_distintas'].max()

# Creamos un DataFrame para mostrar la cantidad de personas con al menos X materias distintas
resultado = pd.DataFrame({
    'materias_minimas': range(1, max_materias + 1),
    'cantidad_personas': [ (materias_por_dni['materias_distintas'] >= x).sum() for x in range(1, max_materias + 1) ]
})

print(resultado)

In [ ]:
total_personas = len(materias_por_dni)

resultado = pd.DataFrame({
    'materias_minimas': range(1, max_materias + 1),
    'cantidad_personas': [ (materias_por_dni['materias_distintas'] >= x).sum() for x in range(1, max_materias + 1) ]
})

resultado['proporcion'] = (resultado['cantidad_personas'] / total_personas) * 100

print(resultado)


In [ ]:
resultado = (
    actas_con_personas.groupby(['dni', 'semestre_relativo'])['materia']
      .nunique()
      .reset_index(name='cantidad_materias_distintas')
)

print(resultado)

# Sumar por DNI para tener total
total_por_dni = resultado.groupby('dni')['cantidad_materias_distintas'].sum().reset_index(name='total_materias_distintas')

# Ahora contar personas que tengan total_materias_distintas >= n
for n in range(1, max(total_por_dni['total_materias_distintas'])):
    cantidad_personas = (total_por_dni['total_materias_distintas'] >= n).sum()
    print(f"Personas con al menos {n} materias distintas (sumando semestres): {cantidad_personas}")

In [ ]:
total_por_dni[total_por_dni['total_materias_distintas'] < 6]

In [ ]:
actas_con_personas[actas_con_personas['dni'] == '19094420'][['fecha', 'semestre_relativo', 'materia', 'resultado', 'nota']]